In [424]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import integrate

In [425]:
data = pd.read_csv('/Users/omarafifi/MyFolders/Differential-Game-Theory-for-SIR-Models/Data/formatted_data.csv')

## Implementing the Adjoint Method

#### 1.1 implement the forward pass 

In [426]:
#step 1 is to solve the forward problem I.e. to integrate the constraining ode. 
timesteps = 21

def SIR(V, t, *args):

    """defines the SIR model
       V = (S, I, R)
       args = (N,Β, γ)
    """

    N, B, g  = args
    S, I, R = V

    dS = -(B/N)*S*I
    dI = (B/N)*S*I - g*I
    dR = g*I

    return dS, dI, dR

def forwardSolve(model, initial_values, params, tf, step):

    """solves the forward problem

       initial_values = (S_0,I_0,R_0)
       params = (N,Β, γ)
       model = SIR
       tf = stop time
       step = number of time steps

    """
    try:
        from scipy.integrate import odeint
        import numpy as np
    except:
        print("unable to install dependancies numpy and scipy.integrate.odeint")
    
    time_steps = np.arange(0, tf, step)
    print(params)

    return odeint(model, t = time_steps, y0 = initial_values, args = params)

In [427]:
gg = forwardSolve(model = SIR, 
             initial_values = (8749080, 455710, 995210),
             params = (10200000,3, 1), 
             tf = 22,
             step = 1)

(10200000, 3, 1)


### 1.2 Backwards integrate  $ \frac{\partial f }{\partial x}  + \lambda (t)^T( \frac{\partial h }{\partial x} - \frac{\partial }{\partial t} \frac{\partial h }{\partial \dot x}) - \frac{\partial \lambda }{\partial t} \frac{\partial h }{\partial \dot x} = 0 $ from $t_f \to 0$

In [428]:
from scipy.interpolate import interp1d

#dummy values for I and I hat 
I = data['I'].to_numpy()
S = data['S'].to_numpy()
I_hat = np.arange(0, 22, 1)

I = interp1d(np.arange(0, 22, 1), I)
S_hat = interp1d(np.arange(0, 22, 1), S, fill_value="extrapolate")
I_hat = interp1d(np.arange(0, 22, 1), I_hat, fill_value="extrapolate")

def lamdaDE(λ, t, *args):
    """
    calculate the derivative of the I component of lambda 
    infections should be a tuple (I, I_pred) of ground truth and predicted values
    over the given time frame. 

    args = (N,Β, γ)
    """

    N, β, γ  = args

    df_dx = 2*np.array([0, I(t) - I_hat(t), 0])

    dh_dx = np.array([[β*I_hat(t)/N ,  β*S_hat(t)/N, 0],
                     [-β*I_hat(t)/N, -β*S_hat(t)/N, 0],
                     [0            ,            -γ, 0]])

    return  df_dx + np.array(λ).T@dh_dx

def backwardsSolve(model, params, tf):
    """Backwards solve for λ(t). 
    """
    #we have to integrate backwards

    from scipy.integrate import odeint
    import numpy as np

    time_steps = np.arange(tf -1, -1, -1)

    return odeint(func = lamdaDE, y0 = [0,0,0], t = time_steps, args = params)


In [429]:
tf = 22
np.arange(tf -1, -1, -1)

array([21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10,  9,  8,  7,  6,  5,
        4,  3,  2,  1,  0])

In [430]:
backwardsSolve(lamdaDE, (100, 3,1), 21)

/var/folders/rs/6g2mhpzn6lq_k8wtscfvn5h80000gn/T/ipykernel_39686/2625914364.py:41: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  return odeint(func = lamdaDE, y0 = [0,0,0], t = time_steps, args = params)


array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-1.06563354e+28, -2.01803050e+33,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00

3) #### Integrate for $d_pF$


$$d_pF(x,p) = \int_0^{t_f} \left ( \frac{\partial f}{\partial p} + \lambda(t)^T  
\frac{\partial h}{\partial p} \right )dt + \lambda(0)^T  \left [ \frac{\partial g}{\partial x_0} \right ]^{-1}
\left [ \frac{\partial g}{\partial p} \right ]$$

In [431]:
df_dp = np.zeros((5))

dg_dx0 = np.eye(3)
dh_dx_dt = np.eye(3)
dg_dp = -np.array([[0,0,1,0,0],
                   [0,0,0,1,0],
                   [0,0,0,0,1]])
dg_dp

N = 1000

def dh_dp(t):
    return np.array([[ I_hat(t)*S_hat(t) ,         0,0,0,0],
                    [-I_hat(t)*S_hat(t)/N,  I_hat(t),0,0,0],
                    [ 0                  , -I_hat(t),0,0,0]])

def integrand(t,λ): 
    return df_dp + λ[t].T@dh_dp(t)


def dF_dp(tf, λ):
 
    #we get all the timesteps: 
    time_steps = np.arange(0, tf, 1)
    integrands = [integrand(t, λ) for t in time_steps]

    #integrate with simpson's method and add the gx(0)gp term 
    return (integrate.simpson(y = integrands, x = time_steps, axis = 0) + 
            λ[0].T@dh_dx_dt@ np.linalg.inv(dg_dx0)@dg_dp)


In [432]:
#set initial paramater values to the data estimates
# (β,γ, N, S_0, I_0, R_0)
p_0 = ( 3, 1, 10200000, 8749080, 455710, 995210)



In [ ]:
from scipy.integrate import odeint, solve_ivp

class solveSIR:

    #class variables 
    S_hat = None
    I_hat = None
    R_hat = None

    #constructor
    def __init__(self, S_true, I_true, R_true, N, p_0):
    
        self.tf = len(S_true)
        self.N = N
        self.p = p_0
        self.time_steps = np.arange(0, len(S), 1)
        self.I = interp1d(self.time_steps,I_true, fill_value="extrapolate")
        self.S = interp1d(self.time_steps,S_true, fill_value="extrapolate")
        self.R = interp1d(self.time_steps,R_true, fill_value="extrapolate")

    def SIR(self, V, t):

        """defines the SIR model dynamics
        V = (S, I, R)
        args = (N,Β, γ)
        """

        B, g  = self.p[0:2]
        S, I, R = V

        dS = -(B/self.N)*S*I
        dI = (B/self.N)*S*I - g*I
        dR = g*I

        return dS, dI, dR

    def forwardSolve(self):

        """solves the forward problem
        """
        return odeint(self.SIR, 
                      y0 = self.p[2:],
                      t = self.time_steps)


    def lamdaDE(self, λ, t):
        """
        calculate the derivative of the I component of lambda 
        infections should be a tuple (I, I_pred) of ground truth and predicted values
        over the given time frame. 
        """
        β, γ  = self.p[0:2]

    

        # simpler names for readability
        I, I_hat, S_hat, N = self.I(t), self.I_hat(t), self.S_hat(t), self.N
 


        df_dx = 2*np.array([0, I - I_hat, 0])
        dh_dx = np.array([[β*I_hat/N  ,  β*S_hat/N, 0],
                          [-β*I_hat/N , -β*S_hat/N, 0],
                          [0          ,         -γ, 0]])

        return  df_dx + np.array(λ).T@dh_dx
    
    def backwardsSolve(self):
        """Backwards solve for λ(t). 
        """
        #we have to integrate backwards

        return odeint(func = self.lamdaDE, 
                      y0 = [0,0,0], 
                      t =self.time_steps[::-1])
    

In [434]:
s = solveSIR(S_true = data['S'],
             I_true = data['I'], 
             R_true = data['R'],
             N =  10200000,
             p_0 = ( 3, 1, 8749080, 455710, 995210))

s.S_hat = S_hat
s.I_hat = I_hat

In [435]:
s.backwardsSolve()

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-9.84282821e+00, -4.03942223e+06,  0.00000000e+00],
       [-6.21118340e+01, -1.66598443e+07,  0.00000000e+00],
       [-2.40795778e+02, -5.71305415e+07,  0.00000000e+00],
       [-8.06795643e+02, -1.89988660e+08,  0.00000000e+00],
       [-2.59013509e+03, -6.39742791e+08,  0.00000000e+00],
       [-8.33215273e+03, -2.22520456e+09,  0.00000000e+00],
       [-2.74885049e+04, -8.08574735e+09,  0.00000000e+00],
       [-9.43889022e+04, -3.10701735e+10,  0.00000000e+00],
       [-3.41818894e+05, -1.27472689e+11,  0.00000000e+00],
       [-1.31232421e+06, -5.57138204e+11,  0.00000000e+00],
       [-5.33955043e+06, -2.60470084e+12,  0.00000000e+00],
       [-2.32718964e+07, -1.33012175e+13,  0.00000000e+00],
       [-1.11090393e+08, -7.63963843e+13,  0.00000000e+00],
       [-5.96215166e+08, -5.05308795e+14,  0.00000000e+00],
       [-3.64391829e+09, -3.87077602e+15,  0.00000000e+00],
       [-2.52892675e+10, -3.43767229e+16